This uses the open AI gym baseline with some slight modifications. Mostly copied code from here
https://github.com/openai/baselines/tree/master/baselines/ppo1. Looks like the version I got from pip and the version currently on master don't quite sync up. (For example there is no tf_util.save_state fcn so we save manually

In [6]:
from baselines.common.cmd_util import make_mujoco_env, mujoco_arg_parser
from baselines.common import tf_util as U
import tensorflow as tf
from baselines import logger
import os
from baselines.ppo1 import mlp_policy, pposgd_simple

import gym
import seagul.envs
import sgillen_research.policies.mlp_relu_policy as mlp_relu_policy


#Needed for saving 
import errno, datetime, time, inspect


def train(env_id, num_timesteps, seed=0):

    U.make_session(num_cpu=16).__enter__()
    
    def policy_fn(name, ob_space, ac_space):
        #return mlp_policy.MlpPolicy(name=name, ob_space=ob_space, ac_space=ac_space, hid_size=64, num_hid_layers=64)
        return mlp_relu_policy.ReluMlpPolicy(name=name, ob_space=ob_space, ac_space=ac_space, hid_size=64, num_hid_layers=64)

    env = gym.make(env_id)
    pi = pposgd_simple.learn(env, policy_fn,
            max_timesteps=num_timesteps,
            timesteps_per_actorbatch=2048,
            clip_param=0.2, entcoeff=0.0,
            optim_epochs=10, optim_stepsize=3e-4, optim_batchsize=64,
            gamma=0.99, lam=0.95, schedule='linear',
        )
    env.close()
   
    return pi



def save_results(filename, description = None):
    """ 
    description: saves the results of a run of the second cell (the one that calls train) in this notebook
    
    """

    save_dir = "data/" + filename + "/"
    os.makedirs(save_dir)
    
    if description is None:
        description  = input("please enter a description of the run")
        
    datetime_str = str(datetime.datetime.today())
    datetime_str = datetime_str.replace(" ", "_")
    
    runtime_str = str(datetime.timedelta(seconds = runtime))
    
    readme = open(save_dir + "README.txt", "w+")
    readme.write("datetime: " + datetime_str + "\n\n")
    readme.write("enviroment: " + env_name + "\n\n")
    readme.write("description: " + description + "\n\n")
    readme.write("time_elapsed: " + runtime_str + "\n\n")
    readme.write("num_timesteps: " + str(num_timesteps) + "\n\n")
    readme.write("seed: " + str(seed) + "\n\n")
    readme.close()

    # TODO add code snippets that correspond to the run
    # TODO somehow store the tensorboard logs here after the fact
    
    saver = tf.train.Saver()
    saver.save(tf.get_default_session(), save_dir + filename)
    
    os.rename("./tmp_logs/", save_dir + "tensorboard")
   
   
env_name = "Acrobot-v1"
#env_name = 'InvertedPendulumPyBulletEnv-v0'
#env_name = "su_cartpole_et-v0"
#env_name = "InvertedDoublePendulum-v2"


In [2]:
start_time = time.time()


num_timesteps = 2e6
seed = 0

logger.configure(dir = "./tmp_logs", format_strs=["tensorboard"] )
with tf.device("/cpu:0"):    
    pi= train(env_name, num_timesteps=num_timesteps, seed=seed)
    
runtime = time.time() - start_time


NameError: name 'time' is not defined

In [ ]:
env = make_mujoco_env(env_name,seed=0)
ob = env.reset()     

while True:
    action = pi.act(stochastic=False, ob=ob)[0]
    ob, _, done, _ =  env.step(action)
    if reward == 1:
        print("balanced")
    #env.render()
    if done:
        ob = env.reset()
        
#U.save_state("./saved/5mil_flat")

In [3]:
#def loaded_policy_fn(name, ob_space, ac_space):
#    return mlp_policy.MlpPolicy(name=name, ob_space=ob_space, ac_space=ac_space, hid_size=64, num_hid_layers=2)

#loader = tf.train.Saver()
#session = loader.restore(tf.get_default_session(), 'saved/test.save')

#env_name = "Acrobot1"

logger.configure()

pi = train(env_name, num_timesteps=1, seed=0)
U.load_state('./acrobot1')
#os.makedirs(os.path.dirname("./saved"), exist_ok=True)
#saver = tf.train.Saver()
#saver.save(tf.get_default_session(), "./saved/test.save")
    
#env = make_mujoco_env('Walker2d-v2',seed=0)
env = gym.make(env_name)
ob = env.reset()     

while True:
    action = pi.act(stochastic=False, ob=ob)[0]
    ob, _, done, _ =  env.step(action)
    env.render()
    if done:
        ob = env.reset()

Logging to /var/folders/qq/gpxz4l6s1tndfdhysbz8bdym0000gn/T/openai-2018-11-17-13-33-33-613084
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
********** Iteration 0 ************
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
    -7.15e-06 |       0.00000 |     197.39848 |      4.18e-05 |       1.09857
     -0.00138 |       0.00000 |     124.14509 |       0.00359 |       1.09511
     -0.00572 |       0.00000 |      92.07592 |       0.01012 |       1.08885
     -0.00517 |       0.00000 |      73.44666 |       0.01165 |       1.08728
     -0.00525 |       0.00000 |      60.46016 |       0.01204 |       1.08678
     -0.00595 |       0.00000 |      50.51894 |       0.01164 |       1.08707
     -0.00655 |       0.00000 |      42.48680 |       0.01282 |       1.08603
     -0.00612 |       0.00000 |      35.85674 |       0.01216 |       1.08658
     -0.00654 |       0.00000 |      30.31948 |       0

ValueError: The passed save_path is not a valid checkpoint: ./acrobot1

In [ ]:
pi.get_variables()
print(logger.dumpkvs())

In [ ]:
mlp_policy.MlpPolicy

In [ ]:
a = {1:'yellow'}

In [ ]:
a[1]